# Project Title Here

**Names**: Aleksandra Grigortsuk, Wendy Zheng, Yinglei Wu, Xiangrui Deng

## 1. Introduction

## 2. Dataset Discussion

## 3. Dataset Cleaning and Exploration

## 4. Basic Descriptive Analytics

## 5. Scaling Decisions

## 6. Clusterability and Clustering Structure

## 7. Clustering Algorithm Selection Motivation

## 8. Clustering Algorithm Practical Considerations

## 9. Clustering Algorithm #1



## 10. Clustering Algorithm #2

## 11. Discussion

## 12. Conclusion

## References